<H1>This notebook will be mainly used for the capstone project</H1>

In [50]:
import pandas as pd
import numpy as np

print ('Hello Capstone Project Course!')

Hello Capstone Project Course!


<H2><font color='blue'>SEGMENTING AND CLUSTERING NEIGHBORHOOD IN TORONTO</font></H2>

<H1>1. DATA COLLECTION</H1>

**Using BeautifulSoup to scrape wikipedia table, to read every row and column and put them in a list (loop, iteratively append values to list)**

In [51]:
from bs4 import BeautifulSoup
import requests
import os
import codecs


wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
header = {'User-Agent': 'Mozilla/5.0'} #Needed to prevent 403 error on Wikipedia


website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')
 

tables = soup.findAll("table", { "class" : "wikitable" })

# show tables
for table in tables:
    print ("###############")
    print (table.text[:100])

for tn in range(len(tables)):
    table=tables[tn]

    # preinit list of lists
    rows=table.findAll("tr")
    row_lengths=[len(r.findAll(['th','td'])) for r in rows]
    ncols=max(row_lengths)
    nrows=len(rows)
    data=[]
    for i in range(nrows):
        rowD=[]
        for j in range(ncols):
            rowD.append('')
        data.append(rowD)
    
    # process html
    for i in range(len(rows)):
        row=rows[i]
        rowD=[]
        cells = row.findAll(["td","th"])
        for j in range(len(cells)):
            cell=cells[j]
            
            #lots of cells span cols and rows so lets deal with that
            cspan=int(cell.get('colspan',1))
            rspan=int(cell.get('rowspan',1))
            for k in range(rspan):
                for l in range(cspan):
                    data[i+k][j+l]+=cell.text
    
data.append(rowD)


###############


Postcode
Borough
Neighbourhood


M1A
Not assigned
Not assigned


M2A
Not assigned
Not assigned


M


<b>
1. Strip out the ''\n'' from the list
2. Convert list to data frame
3. Remove Borough which have values 'Not assigned'
4. Replace Neighbourhood with Borough if neighbourhood value is 'Not assigned'
5. Concatenate Neighbourhood values when under the same postal code
</b>


In [52]:
data_list = [[j.strip('\n') for j in i] for i in data]

df=pd.DataFrame(data_list[1:])
df.columns=['Postcode','Borough','Neighbourhood']

df = df[df.Borough != 'Not assigned']
df.loc[df['Neighbourhood'].eq('Not assigned'), 'Neighbourhood'] = df['Borough']

df=df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()



<H3> Data display </H3>

In [53]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<H2> 2. GET GEOLOCATION DATA FOR EACH POST CODE </H2>
<H3> We want to get geolocation data for each postcode in Toronto. As geocoder is not reliable, we use directly provided Geospatial Coordinates data </H3>

In [54]:
!wget -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv


--2019-02-04 08:43:16--  https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-02-04 08:43:19--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.25.197, 107.152.24.197
Connecting to ibm.box.com (ibm.box.com)|107.152.25.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-02-04 08:43:20--  https://ibm.ent.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.ent.box.com (ibm.ent.box.com)... 107.152.27.211
Connecting to ibm.ent.box.com (ibm.ent.box.com)|107.152.27.211|:443... connected.
HTTP r

In [55]:
df_geo = pd.read_csv('Geospatial_Coordinates.csv')
df_geo.columns=['Postcode','Latitude','Longitude']
df_geo.head()


,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h3> Final result of data frame with geolocation data </h3>

In [56]:
df_join=pd.merge(df,df_geo[['Postcode','Latitude','Longitude']], on='Postcode')
df_join.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<H2>3. Explore and cluster the neighborhoods in Toronto</H2>

Filter only borough of 'Toronto'

In [57]:
toronto_data=df_join[df_join['Borough'].str.contains("Toronto")]


Exploring 'Toronto' Borough. Creating a function that will return nearby venues, sourcing from FourSquare

In [58]:
CLIENT_ID='1UAM34UX4YLZQ2RMVO41SB5WWH5GOZJ5JF3HPZ4TDTTLXEUG'
CLIENT_SECRET='3CB2XHWXIRMTLTXWID4MADTZ5JN5S2ABFEGO10BK3M4KDC3H'
VERSION = '20180605' # Foursquare API version

LIMIT=100
radius=500

In [59]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [60]:
toronto_venues = getNearbyVenues(names=toronto_data['Borough'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )



East Toronto
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
West Toronto
West Toronto
West Toronto
West Toronto
West Toronto
West Toronto
East Toronto


Below are Neighborhoods and its associated Venue details

In [61]:
toronto_venues.head()

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,East Toronto,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
1,East Toronto,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
2,East Toronto,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
3,East Toronto,43.676357,-79.293031,Beaches Dance & Music Studio,43.680595,-79.291300,Dance Studio
4,East Toronto,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [62]:
toronto_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Central Toronto,120,120,120,120,120,120
Downtown Toronto,1281,1281,1281,1281,1281,1281
East Toronto,125,125,125,125,125,125
West Toronto,185,185,185,185,185,185


In [63]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 236 uniques categories.


**To cluster based on venue category, we need to do one-hot encoding of neighborhood by venue categories**

**Let's group by Neighborhood, average count by venue category**

In [64]:
toronto_venues['Borough']

0       East Toronto
1       East Toronto
2       East Toronto
3       East Toronto
4       East Toronto
5       East Toronto
6       East Toronto
7       East Toronto
8       East Toronto
9       East Toronto
10      East Toronto
11      East Toronto
12      East Toronto
13      East Toronto
14      East Toronto
15      East Toronto
16      East Toronto
17      East Toronto
18      East Toronto
19      East Toronto
20      East Toronto
21      East Toronto
22      East Toronto
23      East Toronto
24      East Toronto
25      East Toronto
26      East Toronto
27      East Toronto
28      East Toronto
29      East Toronto
            ...     
1681    West Toronto
1682    West Toronto
1683    West Toronto
1684    West Toronto
1685    West Toronto
1686    West Toronto
1687    West Toronto
1688    West Toronto
1689    West Toronto
1690    West Toronto
1691    West Toronto
1692    West Toronto
1693    West Toronto
1694    East Toronto
1695    East Toronto
1696    East Toronto
1697    East 

In [65]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = toronto_venues['Borough'] 


# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Borough,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**For each neighborhood, top 10 venue category**

In [66]:
toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()
toronto_grouped

,Borough,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,...,0.008333,0.008333,0.000000,0.008333,0.000000,0.008333,0.000000,0.000000,0.000000,0.008333
1,Downtown Toronto,0.000781,0.000781,0.000781,0.000781,0.000781,0.001561,0.001561,0.001561,0.017955,...,0.000781,0.000781,0.002342,0.007806,0.002342,0.005464,0.006245,0.000781,0.001561,0.003123
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024000,...,0.000000,0.008000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032000
3,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005405,...,0.000000,0.000000,0.000000,0.010811,0.000000,0.010811,0.005405,0.000000,0.000000,0.005405


In [67]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [68]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Sushi Restaurant,Restaurant,Dessert Shop,Burger Joint,Pharmacy
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,American Restaurant,Park
2,East Toronto,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Brewery,Café,Park,Yoga Studio,American Restaurant,Pizza Place
3,West Toronto,Bar,Café,Coffee Shop,Restaurant,Italian Restaurant,Pizza Place,Bakery,Men's Store,Park,Bookstore


<font color='blue'>We will cluster neighborhoods based on top 10 venue category</font>

In [69]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 



array([0, 1, 2, 1], dtype=int32)

In [70]:
# add clustering labels
#neighborhoods_venues_sorted=neighborhoods_venues_sorted.drop('Cluster Labels', 1)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Borough'), on='Borough')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Brewery,Café,Park,Yoga Studio,American Restaurant,Pizza Place
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Brewery,Café,Park,Yoga Studio,American Restaurant,Pizza Place
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Brewery,Café,Park,Yoga Studio,American Restaurant,Pizza Place
43,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Brewery,Café,Park,Yoga Studio,American Restaurant,Pizza Place
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Sushi Restaurant,Restaurant,Dessert Shop,Burger Joint,Pharmacy


In [71]:
# create map
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

import folium # map rendering library

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [74]:
import matplotlib.cm as cm
import matplotlib.colors as colors

latitude=43.70011
longitude=-79.4163

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters



<H2>Examining cluster</H2>

In [75]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,0,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Sushi Restaurant,Restaurant,Dessert Shop,Burger Joint,Pharmacy
45,Central Toronto,0,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Sushi Restaurant,Restaurant,Dessert Shop,Burger Joint,Pharmacy
46,Central Toronto,0,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Sushi Restaurant,Restaurant,Dessert Shop,Burger Joint,Pharmacy
47,Central Toronto,0,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Sushi Restaurant,Restaurant,Dessert Shop,Burger Joint,Pharmacy
48,Central Toronto,0,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Sushi Restaurant,Restaurant,Dessert Shop,Burger Joint,Pharmacy
49,Central Toronto,0,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Sushi Restaurant,Restaurant,Dessert Shop,Burger Joint,Pharmacy
63,Central Toronto,0,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Sushi Restaurant,Restaurant,Dessert Shop,Burger Joint,Pharmacy
64,Central Toronto,0,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Sushi Restaurant,Restaurant,Dessert Shop,Burger Joint,Pharmacy
65,Central Toronto,0,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Sushi Restaurant,Restaurant,Dessert Shop,Burger Joint,Pharmacy


In [76]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,American Restaurant,Park
51,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,American Restaurant,Park
52,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,American Restaurant,Park
53,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,American Restaurant,Park
54,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,American Restaurant,Park
55,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,American Restaurant,Park
56,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,American Restaurant,Park
57,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,American Restaurant,Park
58,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,American Restaurant,Park
59,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,American Restaurant,Park


In [77]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Brewery,Café,Park,Yoga Studio,American Restaurant,Pizza Place
41,East Toronto,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Brewery,Café,Park,Yoga Studio,American Restaurant,Pizza Place
42,East Toronto,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Brewery,Café,Park,Yoga Studio,American Restaurant,Pizza Place
43,East Toronto,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Brewery,Café,Park,Yoga Studio,American Restaurant,Pizza Place
87,East Toronto,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Brewery,Café,Park,Yoga Studio,American Restaurant,Pizza Place
